In [24]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision 
import torchvision.transforms as transforms

torch.set_printoptions(linewidth=120)
torch.set_grad_enabled(True)# defult

In [25]:
print(torch.__version__)
print(torchvision.__version__)

1.11.0
0.12.0


In [26]:
def get_num_correct(preds,labels):
    return preds.argmax(dim=1).eq(labels).sum().item()

In [27]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels= 1,out_channels=6,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels= 6,out_channels=12,kernel_size=5)
        
        self.fc1=nn.Linear(in_features = 12*4*4, out_features = 12)
        self.fc2=nn.Linear(in_features = 12, out_features = 60)
        self.out=nn.Linear(in_features = 60, out_features = 10)
        
    def forward(self,t):
        t=t

        t = self.conv1(t)
        t = F.relu(t)
        t = F.max_pool2d(t,kernel_size=2, stride = 2)
        
        t = self.conv2(t)
        t = F.relu(t)
        t = F.max_pool2d(t,kernel_size=2, stride = 2)
        
        t = t.reshape(-1,12*4*4)
        t = self.fc1(t)
        t = F.relu(t)

        t = self.fc2(t)
        t = F.relu(t)

        t = self.out(t)
            
        
        return t
    

In [28]:
train_set = torchvision.datasets.FashionMNIST(
    root='./data/'
    ,train=True
    ,download=True
    ,transform = transforms.Compose([
        transforms.ToTensor()
    ])
)

In [29]:
network = Network()

In [30]:
data_loader = torch.utils.data.DataLoader(
    train_set
    ,batch_size =100
)
batch = next(iter(data_loader))
images,labels = batch

## 计算损失

In [31]:
preds = network(images)
loss = F.cross_entropy(preds,labels)#calculating loss
loss.item()

2.3052875995635986

## 计算梯度

In [32]:
print(network.conv1.weight.grad)

None


In [33]:
loss.backward()

In [34]:
network.conv1.weight.grad.shape

torch.Size([6, 1, 5, 5])

## 更新权重

In [35]:
optimizer= optim.Adam(network.parameters(),lr=0.01)#SGD lr为learning rate

In [36]:
loss.item()

2.3052875995635986

In [37]:
get_num_correct(preds,labels)

9

In [38]:
optimizer.step()

In [39]:
preds = network(images)
loss = F.cross_entropy(preds,labels)

In [40]:
loss.item()

2.2843892574310303

In [41]:
get_num_correct(preds,labels)

10

## 训练单个批次

In [42]:
network = Network()

train_loader = torch.utils.data.DataLoader(train_set,batch_size =100)
optimizer = optim.Adam(network.parameters(),lr = 0.01)

batch = next(iter(train_loader))
images,labels = batch

preds = network(images)
loss = F.cross_entropy(preds,labels)

loss.backward()
optimizer.step()

#-----------------------------------

print('loss:',loss.item())
preds = network(images)
loss = F.cross_entropy(preds,labels)
print('loss2:',loss.item())


loss: 2.299206256866455
loss2: 2.283921718597412
